# SERCC: Temperature records
> This notebook fetches and process data about temperature rankings, percentiles and departure from norm for hundreds of weather stations each day from the [SERCC](https://sercc.oasis.unc.edu/Map.php?region=conus). API feeds [this website](https://sercc.oasis.unc.edu/Map.php?region=conus&), which have many variables (including precipitation) that can offer ideas for adapting the API requests below. 

---

#### Import Python tools and Jupyter config

In [1]:
import us
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from tqdm.notebook import tqdm
from datetime import datetime, timedelta

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None

In [3]:
today = pd.Timestamp("today").strftime("%Y-%m-%d")
yesterday = (pd.Timestamp("today") - timedelta(days=1)).strftime("%Y-%m-%d")

---

## Fetch

In [4]:
# Previously downloaded, Jan. 1, 2023 to May 30, 2024:
# https://ix-cnn-io-prod.s3.amazonaws.com/data/weather/records/sercc_climate_perspectives_records_archive.json

#### Headers for request

In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
}

#### Define the date range

In [6]:
start = "2022-08-01"
start_date = datetime.strptime(start, "%Y-%m-%d")
end = "2024-08-01"
end_date = datetime.strptime(end, "%Y-%m-%d")
date_range = [
    start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)
]
variables = ["mint", "maxt", "avgt", "precip"]

#### Temperature rankings from [SERCC](https://sercc.oasis.unc.edu/Map.php?region=conus)

In [7]:
dfs = []

for var in tqdm(variables):
    for date in tqdm(date_range):
        params = {
            "validdate": date.strftime("%Y-%m-%d"),
            "var": var,
            "thresh": "climper",
            "period": "1_DAY",
            "map_display": "value",  # any map_display_params
            "showthrdx": "true",  # false
            "showcoop": "true",  # false
            "domain": "conus",  # us shows alaska, pr and hawaii
        }

        response = requests.get(
            "https://sercc.oasis.unc.edu/climpermap_json.php",
            params=params,
            headers=headers,
        )

        json_data = response.json()["data"]

        src = (
            pd.DataFrame.from_dict(json_data, orient="index")
            .assign(date=params["validdate"])
            .assign(value_var=var)
            # .query("rank == 1 or rank == '1*'")
            [
                [
                    "id",
                    "city",
                    "state",
                    "value",
                    "value_var",
                    "rank",
                    "ranktext",
                    "dfnlabel",
                    "dfn",
                    "lat",
                    "lon",
                    "date",
                ]
            ]
            .rename(
                columns={
                    "id": "station_id",
                    "dfn": "diff_normal",
                    "dfnlabel": "diff_normal_period",
                }
            )
        ).reset_index(drop=True)
        dfs.append(src)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/732 [00:00<?, ?it/s]

  0%|          | 0/732 [00:00<?, ?it/s]

KeyError: "['id', 'city', 'state', 'value', 'rank', 'ranktext', 'dfnlabel', 'dfn', 'lat', 'lon'] not in index"

In [ ]:
df = pd.concat(dfs)

In [ ]:
df.value_var.value_counts()

In [ ]:
df

---

## Exports

#### JSON

In [ ]:
# df.to_json(
#     "data/processed/sercc_climate_perspectives_records_archive.json",
#     indent=4,
#     orient="records",
#     lines=False,
# )